In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
from __future__ import division

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import xgboost as xgb
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM


# Any results you write to the current directory are saved as output.


# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)

y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values

id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values

# length of dataset
len_train = len(X_train)
len_test  = len(X_test)


In [2]:
X_fit, X_eval, y_fit, y_eval= train_test_split(X_train, y_train, test_size=0.3)

In [ ]:

# classifier
#clf = xgb.XGBClassifier(missing=np.nan, max_depth=5, n_estimators=350, learning_rate=0.03, nthread=4, subsample=0.95, colsample_bytree=0.85, seed=4242)


#clf = OneClassSVM();
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=350)

# fitting
#clf.fit(X_train, y_train, early_stopping_rounds=20, eval_metric="auc", eval_set=[(X_eval, y_eval)])
model.fit(X_train, y_train);
y_eval_pred = model.predict_proba(X_eval);




In [9]:
#print(y_eval_pred[0:500, :])

In [10]:
print('Overall AUC:', roc_auc_score(y_eval,y_eval_pred[:,1]))

Overall AUC: 0.997732798615


In [6]:
%whos

Variable                 Type                      Data/Info
------------------------------------------------------------
OneClassSVM              ABCMeta                   <class 'sklearn.svm.classes.OneClassSVM'>
Pipeline                 type                      <class 'sklearn.pipeline.Pipeline'>
RandomForestClassifier   ABCMeta                   <class 'sklearn.ensemble.<...>.RandomForestClassifier'>
X_eval                   ndarray                   22806x306: 6978636 elems, type `float64`, 55829088 bytes (53.242767333984375 Mb)
X_fit                    ndarray                   53214x306: 16283484 elems, type `float64`, 130267872 bytes (124.23312377929688 Mb)
X_test                   ndarray                   75818x306: 23200308 elems, type `float64`, 185602464 bytes (177.00430297851562 Mb)
X_train                  ndarray                   76020x306: 23262120 elems, type `float64`, 186096960 bytes (177.47589111328125 Mb)
c                        Index                     Index(

In [11]:
from sklearn import metrics

print(metrics.recall_score(y_eval, y_eval_pred, average='binary'))
print(metrics.precision_score(y_eval, y_eval_pred, average='binary'))
print(metrics.f1_score(y_eval, y_eval_pred, average='binary'))

ValueError: Can't handle mix of binary and continuous-multioutput

In [12]:
y_test_pred = model.predict_proba(X_test);
submission = pd.DataFrame({"ID":id_test, "TARGET":y_test_pred[:,1]})
submission.to_csv("submission.csv", index=False)

print('Completed!')

Completed!
